In [ ]:
from repl import *

In [ ]:
#深圳
city='shenzhen'
url='http://apply.sztb.gov.cn/apply/app/status/norm/person'

#杭州
#city='hangzhou'
#url='http://apply.hzcb.gov.cn/apply/app/status/norm/person'

#广州
city='guangzhou'
url='http://apply.gzjt.gov.cn/apply/norm/personQuery.html'

In [ ]:
post_format='pageNo={1}&issueNumber={0}&applyCode='
post_example='pageNo=8&issueNumber=201607&applyCode='

In [ ]:
issue= new_spider('issue')

In [ ]:
_issues=issue.visit(url,post_data=post_example).py_query()('#issueNumber')
issues= [r.text for r in _issues.children()]

In [ ]:
s=new_spider('list')

In [ ]:
s.visit(url, post_data=post_example) \
        .search_xpath('\d{13}', 'id',mode='re') \
        .search_xpath('^杨[\u4e00-\u9fa5]{2}$', 'name',mode='re').accept().test().get()

In [ ]:
c=get_default_connector()

In [ ]:
t=new_task('main')

In [ ]:
t.clear()
t.pyge('issue', script=issues)
t.merge('issue:post', script=post_format, merge_with='2')
t.addnew('url', value=url)
t.tolist()
t.crawler('url', selector='issue', post_data='[post]')
t.xpath('Content:page',script='/html/head/script[2]',mode='html')
t.html('page',mode='decode')
t.regex('page',script="'\d+'",index=2)
t.number('page')
t.delete('Content post')
t.rangege('p',max='[page]',mode='cross')
t.merge('issue:post',script=post_format,merge_with='p')
t.crawler('url',selector='list',post_data='[post]',new_col='issue')
t.dbex('id',connector='mongo',table='%s_yaohao'%(city))
t.get(etl_count=100)

In [ ]:
t.distribute()

In [ ]:
t.stop_server()